In [ ]:
import time
import cv2
import numpy as np
import tensorflow as tf
import math
import socket

In [ ]:
def calculate_slope(x1, y1, x2, y2):
    return (y2 - y1) / (x2 - x1) if x2 - x1 != 0 else math.inf

In [ ]:
def calculate_angle(x1, y1, x2, y2):
    slope = calculate_slope(x1, y1, x2, y2)

    if slope == math.inf:
        return 90.0
    else:
        angle_rad = math.atan(slope)
        angle_deg = math.degrees(angle_rad)
        return angle_deg

In [ ]:
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
serverAddressPort = ("127.0.0.1", 5052)

interpreter = tf.lite.Interpreter(model_path="mpii_resnet_100epoch_bin_tflite.tflite")
interpreter.allocate_tensors()

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 360)
pTime = 0

movement_tracked = -2

In [ ]:
while True:
    success, imgRGB = cap.read()
    resized = tf.image.resize(imgRGB, (108, 192))

    input_tensor = resized / 255.0
    input_tensor = np.expand_dims(input_tensor, axis=0)

    interpreter.set_tensor(interpreter.get_input_details()[0]['index'], input_tensor)

    interpreter.invoke()

    yhat = interpreter.get_tensor(interpreter.get_output_details()[0]['index'])

    s_coords = yhat[0]
    s_img = np.copy(imgRGB)
    keys_coords = list(np.multiply(list(zip(s_coords[::2], s_coords[1::2])), [640, 360]))

    for i, (x, y) in enumerate(keys_coords):
        cv2.circle(s_img, (int(x), int(y)), 8, (255, 0, 0), -1)
        if i < len(keys_coords) - 1:
            cv2.line(s_img, (int(x), int(y)), (int(keys_coords[i + 1][0]), int(keys_coords[i + 1][1])), (0, 255, 0), 3)

    x1, y1 = map(int, keys_coords[0])
    x2, y2 = map(int, keys_coords[1])

    angle = calculate_angle(x1, y1, x2, y2)

    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    s_img = cv2.flip(s_img, 1)
    cv2.putText(s_img, str(int(fps)), (70, 70), cv2.FONT_HERSHEY_DUPLEX, 3, (255, 0, 0), 3)

    old = movement_tracked

    if -80 < angle < 0:
        movement_tracked = -1
    elif 80 > angle > 0:
        movement_tracked = 1
    else:
        movement_tracked = 0

    sock.sendto(str.encode(str(movement_tracked)), serverAddressPort)

    cv2.imshow("Video", s_img)
    cv2.waitKey(1)